In [2]:
from langchain import hub
# obj = ChatPromptTemplate()
obj = hub.pull("hardkothari/prompt-maker")

In [3]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
# prompt = llm(obj.format_messages(task="search and summarize artificial intelligence research paper", lazy_prompt="you are a expert about searching and summarize about research paper"))
prompt = llm(obj.format_messages(task="Extract the name of a research topic or research paper", lazy_prompt="Extract a research topic or research paper name from a given user input. Do not return any words other than the topic or name"))

print(prompt.content)

"""
As an expert in researching and summarizing research papers, your task is to conduct a comprehensive search on a given topic and provide a concise summary of the key findings and insights from the selected papers. Your summary should include an overview of the research question, methodology, results, and conclusions of each paper. Additionally, please organize the summaries in a logical and coherent manner, highlighting any common themes or patterns that emerge from the research. Aim to provide a well-structured and informative summary that can serve as a valuable resource for readers seeking a quick understanding of the current state of research on the topic.
"""
